# Final LDA Model

In [7]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

from datetime import datetime

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
# import pyLDAvis.gensim  # Author: don't skip this
# pyLDAvis.gensim.prepare

# I think i need another one:
import pyLDAvis.gensim_models
import pyLDAvis.gensim_models as gensimvis

# Plots
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# TF.IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_distances

In [8]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')
stop_words.extend(['tenlastelegging', 'hof', 'althans', 'tenlastegelegd', 'naan', 'verklaring', 'verklaren', 'benadelen', 'naam', 'aangeefster', 'aangever', 'aangev', 'verbalisant', 'slachtoffer', 'rechtbank', 'uur', 'uren', 'weten', 'bestaan', 'waarheid', 'daarvoor', 'genaamd', 'maken', 'gaan', 'toverweging', 'aanzien', 'bewijs', 'feit', 'grond', 'staan', 'vaststellen', 'halen', 'vervolgens', 'nemen', 'aanhouden', 'bevinden', 'officier', 'justitie', 'overtuigen', 'bewijzen', 'maken', 'stellen', 'leggen', 'dienen', 'vrijspreken', 'daarnaast', 'bezigen', 'willen', 'gaan', 'vervolgens', 'raken', 'weten', 'proberen', 'echter', 'vraag', 'verdenken', 'vervatten', 'beslissing', 'hoger_beroep', 'verkort_vonni', 'geacht', 'instellen', 'ander', 'zien', 'toebehoren', 'hoeveelheid', 'lijst_ii', 'bereiken'])

# Data 1

In [13]:
df = pd.read_csv('/Users/dj/Python - UvA/DSP/data/df_final.csv', index_col=6, parse_dates=['date']) 

df_be = df[df['bewijs'] != '[]'] #bewijs
df_te = df[df['tll'] != '[]'] #tenlastelegging

df['bete'] = df['bewijs'] + df['tll'] 
df = df[df['bete'] != '[][]']
df = df.drop(labels=['bewijs', 'tll'], axis=1)

# ---- Crime type

harddrugs_pattern = 'harddrugs|hard drugs'
inbraak_pattern = 'inbraak|huisbraak|inbreken'
mishandeling_pattern = 'mishandeling|mishandelen'
verkrachting_pattern = 'verkrachting|verkrachten|verkracht'

df2 = df.dropna(subset = ['inhoudsindicatie'])
print(df2.shape)
df_inbraak = df2[df2['inhoudsindicatie'].str.contains(inbraak_pattern)]
print(df_inbraak.shape)
df_harddrugs = df2[df2['inhoudsindicatie'].str.contains(harddrugs_pattern)]
print(df_harddrugs.shape)
df_mishandeling = df2[df2['inhoudsindicatie'].str.contains(mishandeling_pattern)]
print(df_mishandeling.shape)
df_verkrachting = df2[df2['inhoudsindicatie'].str.contains(verkrachting_pattern)]
print(df_verkrachting.shape)


(19966, 8)
(520, 8)
(334, 8)
(2417, 8)
(428, 8)


In [14]:
df.head()

,Unnamed: 0,filename,ecli,subject,spatial,case_nr,inhoudsindicatie,bete
date,,,,,,,,
2018-01-17,9,ECLI_NL_GHAMS_2018_104.xml,ECLI:NL:GHAMS:2018:104,Strafrecht,Amsterdam,23-001099-16.a,\nVrijspraak wegens onvoldoende overtuigend be...,[][' tenlastelegging aan de verdachte is ten...
2018-01-17,10,ECLI_NL_GHAMS_2018_105.xml,ECLI:NL:GHAMS:2018:105,Strafrecht,Amsterdam,23-002194-17.a,\nVeroordeling wegens winkeldiefstal met recid...,[][' tenlastelegging aan de verdachte is ten...
2018-03-30,16,ECLI_NL_GHAMS_2018_1064.xml,ECLI:NL:GHAMS:2018:1064,Strafrecht,Amsterdam,23-002597-17,\nbelediging\n,[][' tenlastelegging aan de verdachte is ten...
2018-03-30,17,ECLI_NL_GHAMS_2018_1065.xml,ECLI:NL:GHAMS:2018:1065,Strafrecht,Amsterdam,23-002801-16,\ndiefstal\n,[][' tenlastelegging aan de verdachte is ten...
2018-03-30,19,ECLI_NL_GHAMS_2018_1072.xml,ECLI:NL:GHAMS:2018:1072,Strafrecht,Amsterdam,23-002206-17.a,\nDe verdachte wordt veroordeeld wegens het be...,[' bewijsmiddelen nu de verdachte ter terec...


# Data 2

In [ ]:

''''
df = pd.read_csv('/Users/dj/Python - UvA/DSP/data/dataset_18_21.csv', index_col=5, parse_dates=['date']) 
df = df.drop(labels=['ecli', 'case_nr', 'bewijs'], axis=1)
df = df.rename(columns={"tll": "bete"})
df.head()
'''

In [15]:
df18 = df.loc["2018-01-01 00:00:00":"2018-12-31 00:00:00"]
df19 = df.loc["2019-01-01 00:00:00":"2019-12-31 00:00:00"]
df20 = df.loc["2020-01-01 00:00:00":"2020-12-31 00:00:00"]
df21 = df.loc["2021-01-01 00:00:00":"2021-12-31 00:00:00"]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated 

In [16]:
print(df18.shape)
print(df19.shape)
print(df20.shape)
print(df21.shape)

(5062, 8)
(5076, 8)
(4961, 8)
(4877, 8)


# Build Giant Function

In [ ]:

def get_lda_model(df):

    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

    data_words = list(sent_to_words(df['bete']))

        # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    nlp = spacy.load("nl_core_news_sm")

    def sent_to_words(sentences):
        for sentence in sentences:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

    # Define functions for stopwords, bigrams, trigrams and lemmatization
    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]

    def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
        """https://spacy.io/api/annotation"""
        texts_out = []
        for sent in texts:
            doc = nlp(" ".join(sent)) 
            texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        return texts_out



    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    # nlp = spacy.load('nl', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    data_lemmatized = remove_stopwords(data_lemmatized)

    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    # corpus1 = [str(item) for item in corpus]

    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=5, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)


    ######## TF.IDF ########

    dat_lem_untok = [" ".join(x) for x in data_lemmatized]
    tfidf_vectorizer = TfidfVectorizer(input=dat_lem_untok, stop_words=stop_words)
    tfidf_vector = tfidf_vectorizer.fit_transform(dat_lem_untok)
    tfidf_df = pd.DataFrame(tfidf_vector.toarray(), columns=tfidf_vectorizer.get_feature_names())
    tfidf_df = tfidf_df.stack().reset_index()
    tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})
    top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

     

    return(lda_model, data_lemmatized, corpus, id2word, top_tfidf)

In [ ]:
d = [df18, df19, df20, df21]
q = 1
lda_models = []
dat_lems = []
corps = []
ids = []
top_idfs = []

for i in d:
    print(q)
    q += 1
    l, dl, c, i2words, t = get_lda_model(i)
    lda_models.append(l)
    dat_lems.append(dl)
    corps.append(c)
    ids.append(i2words)
    top_idfs.append(t)



# Compute Model Perplexity and Coherence Score

In [ ]:
# 18 = 0 / 19 = 1 / 20 = 2 / 21 = 3

a = 3

# Compute Perplexity
print('\nPerplexity: ', lda_models[a].log_perplexity(corps[a]))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_models[a], texts=dat_lems[a], dictionary=ids[a], coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics-keywords

In [ ]:
# 18 = 0 / 19 = 1 / 20 = 2 / 21 = 3

a = 0

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_models[a], corps[a], ids[a], R = 10)
vis

# TF.IDF 

In [ ]:
test = top_idfs[0].sort_values(by=['tfidf'], ascending=[False]).head(20)

In [ ]:
top_idfs[0].sort_values(by=['tfidf'], ascending=[False]).head(20)

In [ ]:
top_idfs[3][top_idfs[3]['term'].str.contains('hof')]

In [ ]:
values = []
years = ('2018', '2019', '2020', '2021')
term = 'kledingwinkel'

def avg_terms(data, term):
    avg = data[data['term'].str.contains(term)]['tfidf'].mean()
    return(avg)
    
for i in top_idfs:
    if term in list(i['term']):
        a  = avg_terms(i, term)
        values.append(a)
    else:
        values.append(0)

In [ ]:
x_pos = [0,1,2,3]

figure(figsize=(12, 10), dpi=80)

plt.plot(values)


# Add title and axis names
plt.title('Average TF-IDF Scores of term: {}'.format(term))
plt.xlabel('Years')
plt.ylabel('TF.IDF Scores')
 
# Create names on the x axis
plt.xticks(x_pos, years)
 
# Show graph
plt.show()

In [ ]:
values

In [ ]:
import altair as alt
import numpy as np

# 18 = 0 / 19 = 1 / 20 = 2 / 21 = 3

a = 0

# Terms in this list will get a red dot in the visualization
term_list = ['kledingwinkel', 'albert_heijn'] # Highlight the words of interest

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_idfs[a].copy()
top_tfidf_plusRand = top_tfidf_plusRand.iloc[:500,]
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf_plusRand.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 1200)

# Scattertext

For some reason, this plot does not work in visual studio code. One has to use the jupyter notebook in the safari browser. At least thats the case for me. 

In [ ]:
import scattertext as st
import re, io
from pprint import pprint
import pandas as pd
import numpy as np
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
import scattertext as st
import re, io
from pprint import pprint
import pandas as pd
import numpy as np
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
import selenium
from selenium import webdriver
import webbrowser
import codecs
display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
d0 = d[0]
d1 = d[1]

d[0]['lemm'] = [" ".join(x) for x in dat_lems[0]]
d[1]['lemm'] = [" ".join(x) for x in dat_lems[1]]

d0['year'] = pd.to_datetime(d0.index).year
d1['year'] = pd.to_datetime(d1.index).year

d0['year'] = d0['year'].astype("string")
d1['year'] = d1['year'].astype("string")

frames = [d0, d1]

result = pd.concat(frames)

nlp = spacy.load("nl_core_news_sm")

In [ ]:
result['parsed'] = result.lemm.apply(nlp)

In [ ]:
result = result.rename(columns={"Unnamed: 0": "Doc_Number"})
result["Doc_Number"] = result["Doc_Number"].astype("category")
result["year"] = result["year"].astype("category")

In [ ]:
corpus_scat_text = st.CorpusFromParsedDocuments(result, category_col='year', parsed_col='parsed').build()

In [ ]:
html = produce_scattertext_explorer(corpus_scat_text,
                                    category='2018',
                                    category_name='2018',
                                    not_category_name='2019',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    metadata=corpus_scat_text.get_df()['filename'],
                                    transform= st.Scalers.log_scale_standardize) 


In [ ]:
import codecs
import webbrowser
import IPython

file_name = 'result_scatter_scale.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [ ]:
# Trying so use data_lemmatized as input
test = df.iloc[0:10, :]

In [ ]:
test['test'] = test.bete.apply(nlp)

In [ ]:
test.head()

In [ ]:
type(test['bete'][0])

In [ ]:
type(test['test'][0])

In [ ]:
d[0].shape

In [ ]:
len(dat_lems[3])

In [ ]:
d[0]['lemm'] = dat_lems[0]

In [ ]:
d[0].head()

In [ ]:
dat_lem_untok = [" ".join(x) for x in dat_lems[0]]

In [ ]:
len(dat_lem_untok)

In [ ]:
type(d[0]['lemm'][0][0])

In [ ]:
'''
d 
q 
lda_models 
dat_lems 
corps 
ids 
top_idfs 
'''

In [ ]:
d[0]

In [ ]:
top_idfs[0]

In [ ]:
test = pd.read_csv('/Users/dj/Python - UvA/DSP/data_2018.csv', index_col=0) 

In [ ]:
type(test.index[0])